In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score
import joblib

# Load the data
train_labels = pd.read_csv('/content/training_set_labels.csv')
train_features = pd.read_csv('/content/training_set_features.csv')
test_features = pd.read_csv('/content/test_set_features.csv')
submission_format = pd.read_csv('/content/submission_format.csv')

# Merge labels with features for the training set
train_data = pd.merge(train_features, train_labels, on='respondent_id')

# Prepare training data
X_train = train_data.drop(columns=['respondent_id', 'xyz_vaccine', 'seasonal_vaccine'])
y_train = train_data[['xyz_vaccine', 'seasonal_vaccine']]

# Prepare test data
X_test = test_features.drop(columns=['respondent_id'])

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = MultiOutputClassifier(RandomForestClassifier(random_state=42))

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict probabilities on the test data
test_preds = pipeline.predict_proba(X_test)
test_preds = np.column_stack([pred[:, 1] for pred in test_preds])  # Get probabilities for class 1

# Create the submission file
submission_data = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_preds[:, 0],
    'seasonal_vaccine': test_preds[:, 1]
})
submission_data.to_csv('/content/submission_predictions.csv', index=False)

# Save the trained pipeline for future use
joblib.dump(pipeline, '/content/trained_pipeline.pkl')

print("Predictions saved to submission_predictions.csv")


Predictions saved to submission_predictions.csv
